In [1]:

from __future__ import print_function

import sys

sys.path.extend(['/home/jorg/repository/dcnn_acdc'])

import matplotlib
import matplotlib.pyplot as plt

from matplotlib import cm


from datetime import datetime
from pytz import timezone
import itertools
import copy

import SimpleITK as sitk
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn import init

import numpy as np
import dill
import os
import glob
from collections import OrderedDict

from in_out.load_data import ACDC2017DataSet
from config.config import config
from utils.experiment import Experiment, ExperimentHandler
from utils.batch_handlers import TwoDimBatchHandler
from utils.test_handler import ACDC2017TestHandler, test_ensemble
from utils.test_results import TestResults
from plotting.learning_curves import loss_plot
from plotting.uncertainty_plots import analyze_slices
from plotting.slice_outliers_plots import plot_outlier_slice_hists
from utils.generate_uncertainty_maps import ImageOutliers

from models.dilated_cnn import BaseDilated2DCNN

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [5]:
ROOT_DIR = "/home/jorg/repository/dcnn_acdc"
LOG_DIR = os.path.join(ROOT_DIR, "logs")
CYCLE_OFFSET = 100000
# set seeds
SEED = 4325
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.enabled = True
np.random.seed(SEED)

# IMPORTANT - THE FOLD WE'RE USING FOR TESTING. NOTE: Training and testing fold should be the same
# we raise an error in the test_ensemble method if this is not the case!
TEST_FOLD = 2

# baseline: 
exp_base = {2: "20180328_10_53_16_dcnnv1_150000E_lr2e02",
            0: "20180330_09_56_39_dcnnv1_150000E_lr2e02"}
exp_base_path = os.path.join(LOG_DIR, exp_base[TEST_FOLD])

exp_mc005_path = None
# MC dropout models
# Fold 2: 
# Fold 0: 20180413_16_36_58_dcnn_mcv1_150000E_lr2e02
# exp_mc005 = {2: "",
#             0: "20180413_16_36_58_dcnn_mcv1_150000E_lr2e02"}
# exp_mc005_path = os.path.join(LOG_DIR, exp_mc005[TEST_FOLD])

# MC dropout models
# Fold 2: 
# Fold 0: 20180413_16_36_58_dcnn_mcv1_150000E_lr2e02
exp_mc01 = {2: "20180420_11_59_43_dcnn_mcv1_150000E_lr2e02",
            0: "20180416_08_25_43_dcnn_mcv1_150000E_lr2e02"}
exp_mc01_path = os.path.join(LOG_DIR, exp_mc01[TEST_FOLD])

exp_mc005_expert1_path = None
# This model was retrained on checkpoint 140000 of the baseline MC model 20180413_16_36_58_dcnn_mcv1_150000E_lr2e02
# @epoch0-> generate uMaps, trained for 20,000 epochs where uMaps are re-generated each 10,000 iterations
# IMPORTANT: the model was ONLY trained on outlier slices for 20,000 epochs (after pre-training of 140,000)
# Fold 2: 
# Fold 0: 20180413_17_03_22_dcnn_mcv1_150000E_lr2e02
# exp_mc005_expert1 = {2: "",
#                      0: "20180417_17_43_20_dcnn_mcv1_20000E_lr2e02"}
# exp_mc005_expert1_path = os.path.join(LOG_DIR, exp_mc005_expert1[TEST_FOLD])

# Fold 2: 
# Fold 0: 20180418_15_02_05_dcnn_mcv1_150000E_lr2e02
exp_mc01_brier = {2: "20180420_12_00_23_dcnn_mcv1_150000E_lr2e04",
                  0: "20180418_15_02_05_dcnn_mcv1_150000E_lr2e02"}
exp_mc01_brier_path = os.path.join(LOG_DIR, exp_mc01_brier[TEST_FOLD])

In [7]:
exper_base = ExperimentHandler.load_experiment(exp_base_path)
exper_hdl_base = ExperimentHandler(exper_base, use_logfile=False)
exper_hdl_base.set_root_dir(ROOT_DIR)
exper_hdl_base.set_model_name("DCNN baseline fold-{}".format(exper_hdl_base.exper.run_args.fold_ids[0]))
print("Model name {}".format(exper_hdl_base.exper.model_name))

if exp_mc005_path is not None:
    exper_mc005 = ExperimentHandler.load_experiment(exp_mc005_path)
    exper_hdl_mc005 = ExperimentHandler(exper_mc005, use_logfile=False)
    exper_hdl_mc005.set_root_dir(ROOT_DIR)
    exper_hdl_mc005.set_model_name("MC-dropout fold-{} (p={})".format(exper_hdl_mc005.exper.run_args.fold_ids[0],
        exper_hdl_mc005.exper.run_args.drop_prob))
    print("Model name {}".format(exper_hdl_mc005.exper.model_name))

exper_mc01 = ExperimentHandler.load_experiment(exp_mc01_path)
exper_hdl_mc01 = ExperimentHandler(exper_mc01, use_logfile=False)
exper_hdl_mc01.set_root_dir(ROOT_DIR)
exper_hdl_mc01.set_model_name("MC-dropout fold-{} (p={})".format(exper_hdl_mc01.exper.run_args.fold_ids[0],
    exper_hdl_mc01.exper.run_args.drop_prob))
print("Model name {}".format(exper_hdl_mc01.exper.model_name))

if exp_mc005_expert1_path is not None:
    exper_mc005_expert1 = ExperimentHandler.load_experiment(exp_mc005_expert1_path)
    exper_hdl_mc005_expert1 = ExperimentHandler(exper_mc005_expert1, use_logfile=False)
    exper_hdl_mc005_expert1.set_root_dir(ROOT_DIR)
    exper_hdl_mc005_expert1.set_model_name("MC-dropout-expert1 fold-{} (p={})".format(
                                         exper_hdl_mc005_expert1.exper.run_args.chkpnt_freq,
                                         exper_hdl_mc005_expert1.exper.run_args.fold_ids[0],
                                         exper_hdl_mc005_expert1.exper.run_args.drop_prob))
    print("Model name {}".format(exper_hdl_mc005_expert1.exper.model_name))

exper_mc01_brier = ExperimentHandler.load_experiment(exp_mc01_brier_path)
exper_hdl_mc01_brier = ExperimentHandler(exper_mc01_brier, use_logfile=False)
exper_hdl_mc01_brier.set_root_dir(ROOT_DIR)
exper_hdl_mc01_brier.set_model_name("MC-dropout brier fold-{} (p={})".format(
                                exper_hdl_mc01_brier.exper.run_args.fold_ids[0],
                                exper_hdl_mc01_brier.exper.run_args.drop_prob))
print("Model name {}".format(exper_hdl_mc01_brier.exper.model_name))

try:
    del test_set
except:
    pass

test_set = ACDC2017TestHandler(exper_config=exper_hdl_mc01.exper.config, 
                               search_mask=config.dflt_image_name + ".mhd", fold_ids=[TEST_FOLD],
                               debug=False, batch_size=25, use_cuda=True, load_train=False, load_val=True,
                               use_iso_path=True)

  4%|▍         | 1/25 [00:00<00:03,  7.70it/s]

Load experiment from /home/jorg/repository/dcnn_acdc/logs/20180328_10_53_16_dcnnv1_150000E_lr2e02/stats/exper_stats.dll
Model name DCNN baseline fold-2
Load experiment from /home/jorg/repository/dcnn_acdc/logs/20180420_11_59_43_dcnn_mcv1_150000E_lr2e02/stats/exper_stats.dll
Model name MC-dropout fold-2 (p=0.1)
Load experiment from /home/jorg/repository/dcnn_acdc/logs/20180420_12_00_23_dcnn_mcv1_150000E_lr2e04/stats/exper_stats.dll
Model name MC-dropout brier fold-2 (p=0.1)
INFO - Testhandler - >>> Search in val-dir for /home/jorg/repository/dcnn_acdc/data/Folds/fold2/validate/images_iso/*patient*.mhd <<<
INFO - File list contains 50 files, hence 25 patients


100%|██████████| 25/25 [00:01<00:00, 16.69it/s]

INFO - Successfully loaded 25 ED/ES patient pairs


In [ ]:
settings = {'sample_eval': {},
            'ensemble_eval': {"mc_samples": 1}, "sample_weights": False, "store_details": False,
           'generate_stats': False, "save_results": True, "checkpoints": [150000],
           'use_uncertainty': False, "discard_outliers": False}


mc_samples = 1
sample_weights = False
store_details = True
image_range = [0, 24]
if not sample_weights:
    generate_stats = False
else:
    generate_stats = True
save_results = False
checkpoints = [150000] #  [150000]
use_uncertainty = True
referral_threshold = 0.25
discard_outliers = False

exper_handler = exper_hdl_mc005_brier

exper_handler.logger = None   
exper_handler.exper.outliers_per_epoch = None

test_ensemble(test_set, exper_handler, mc_samples=mc_samples, sample_weights=sample_weights,  
              discard_outliers=discard_outliers, referral_threshold=referral_threshold, image_range=image_range, 
              checkpoints=checkpoints, use_uncertainty=use_uncertainty,
              verbose=False, reset_results=True, store_details=store_details, 
              generate_stats=generate_stats, save_results=save_results, do_filter=True)
# do_filter = 6 most connected component filtering


    
print("Ready")

In [28]:
exper_handler.test_results.compute_mean_stats()
exper_handler.test_results.show_results()


------------------------------------------------------------------------------------------------
Overall:	dice(RV/Myo/LV): ES 0.79 (0.01)/0.79 (0.00)/0.92 (0.03)	ED 0.89 (0.03)/0.78 (0.02)/0.94 (0.00)
				 ES 0.85 (0.01)/0.84 (0.00)/0.94 (0.01)	ED 0.91 (0.03)/0.84 (0.01)/0.96 (0.00)
Hausdorff(RV/Myo/LV):	ES 17.20 (3.99)/27.59 (12.51)/12.05 (0.85)	ED 21.00 (0.33)/26.65 (16.55)/9.39 (3.13)
After referral:		ES 16.24 (4.95)/25.57 (12.51)/9.95 (2.95)	ED 15.62 (4.77)/25.85 (15.75)/9.06 (3.46)
Ref ES %: 0.41 (std=0.34,med=0.28)	Red%: 28.71 (std=21.28,med=24.18)	PR/RC 0.25 (0.23)/0.29 (0.21)
Ref ED %: 0.58 (std=0.52,med=0.40)	Red%: 25.30 (std=15.42,med=22.67)	PR/RC 0.17 (0.12)/0.23 (0.10)


In [89]:
mc_probs = exper_handler.test_results.mc_pred_probs[0]
std_stats = exper_handler.test_results.stddev_maps[0]
print(std_stats.shape)
sliceid = 7
c=1  # ES-RV
# get probs from slice 8 [#samples, classes, width, height, #slices]
mc_probs_slice = mc_probs[:, 1, :, :, 7]
# get stddev-maps [classes, width, height, #slices]
std_stats_slice = std_stats[1, :, :, 7]
print(std_stats_slice.shape)
print(mc_probs_slice.shape)

(8, 223, 264, 8)
(223, 264)
(10, 223, 264)


In [195]:
exper_handler = exper_hdl_mc01_brier
t_path = "/home/jorg/repository/dcnn_acdc/logs/20180418_15_02_05_dcnn_mcv1_150000E_lr2e02//stats/test_results_25imgs_mc10_fold0_ep150000.dll"
test_results = TestResults.load_results(t_path)
print("Num of images in test set {}".format(len(test_results.image_ids)))
exper_handler.test_results = test_results

INFO - Loading results from file /home/jorg/repository/dcnn_acdc/logs/20180418_15_02_05_dcnn_mcv1_150000E_lr2e02//stats/test_results_25imgs_mc10_fold0_ep150000.dll
INFO - Successfully loaded TestResult object.
Num of images in test set 25


In [ ]:
image_range = None
u_type = "stddev"

analyze_slices(exper_handler, image_range=image_range, do_save=True, do_show=False, u_type=u_type,
              use_high_threshold=True, translate_img_range=True)

In [9]:
# image_range = [16,17, 18, 19]
errors_only = False

for image_num in image_range:
    exper_handler.test_results.visualize_uncertainty_histograms(image_num=image_num, width=20, height=60, 
                                                                info_type="uncertainty",
                                                                 do_save=True, slice_range=None,
                                                                 std_threshold=0., errors_only=errors_only, 
                                                                 do_show=False, use_bald=True,
                                                                 model_name = exper_handler.exper.model_name)
                                                                 # model_name="MC-dropout (0.1)") [0,1,2,3,4,5]
print("INFO - Ready!")

Rows/columns 14/4
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180416_08_25_43_dcnn_mcv1_150000E_lr2e02/figures/patient080/analysis_seg_err_slice1_mc10.pdf
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180416_08_25_43_dcnn_mcv1_150000E_lr2e02/figures/patient080/analysis_seg_err_slice2_mc10.pdf
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180416_08_25_43_dcnn_mcv1_150000E_lr2e02/figures/patient080/analysis_seg_err_slice3_mc10.pdf
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180416_08_25_43_dcnn_mcv1_150000E_lr2e02/figures/patient080/analysis_seg_err_slice4_mc10.pdf
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180416_08_25_43_dcnn_mcv1_150000E_lr2e02/figures/patient080/analysis_seg_err_slice5_mc10.pdf
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180416_08_25_43_dcnn_mcv1_150000E_lr2e02/figures/patient080/analysis_seg_err_slice6_mc10.pdf
Rows/colum

INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180416_08_25_43_dcnn_mcv1_150000E_lr2e02/figures/patient099/analysis_seg_err_slice12_mc10.pdf
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180416_08_25_43_dcnn_mcv1_150000E_lr2e02/figures/patient099/analysis_seg_err_slice13_mc10.pdf
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180416_08_25_43_dcnn_mcv1_150000E_lr2e02/figures/patient099/analysis_seg_err_slice14_mc10.pdf
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180416_08_25_43_dcnn_mcv1_150000E_lr2e02/figures/patient099/analysis_seg_err_slice15_mc10.pdf
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180416_08_25_43_dcnn_mcv1_150000E_lr2e02/figures/patient099/analysis_seg_err_slice16_mc10.pdf
Rows/columns 14/4
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180416_08_25_43_dcnn_mcv1_150000E_lr2e02/figures/patient100/analysis_seg_err_slice1_mc10.pdf
INFO 

In [43]:
exper_handler = exper_hdl_mc005
exper_handler.get_outlier_stats(verbose=True)

Loaded outlier stats for 1 training epochs (property=outliers_per_epoch, dictionary (key=epoch) with tuple (6 elements) and property=test_results_per_epoch, dict (key=epoch) with tuple (4))


In [44]:

image_num = 5
# geth the patient id for this image_num
patient_id = test_set.img_file_names[image_num]
print("Patient ID {}".format(patient_id))
# get the "latest" (max epoch) outlier statistics, tuple with 6 dictionaries
outlier_stats = exper_handler.exper.outliers_per_epoch[exper_handler.exper.epoch_id]
image_outliers = ImageOutliers(patient_id, outlier_stats)
print(image_outliers.out_img_cls_es)
print(image_outliers.out_img_cls_ed)

Patient ID patient036
{7: [1, 2, 3]}
{6: [1], 7: [1, 2, 3]}
